In [1]:
# dirs
webdriver_dir = "C:\\Users\\elena.khotlyannik\\Downloads\\chromedriver_win32\\chromedriver" # куда установить вед драйвер
downloads_dir = "C:\\Users\\elena.khotlyannik\\Downloads" #путь к папке "Загрузки" - туда автоматически грузятся файлы (потом, ниже по коду, очищаю директорию)
acids_sequences = 'C:\\Users\\elena.khotlyannik\\Documents\\HSE\\project\\pdb_seqres.txt'# нужно предварительно положить сюда файл pbd_seqres.txt
fileformapp ='C:\\Users\\elena.khotlyannik\\Documents\\HSE\\project\\codon mapping.txt'

In [ ]:
#!pip install selenium

#также предварительно скаичваю файл chromedriver_win32 из 
# https://chromedriver.storage.googleapis.com/index.html?path=101.0.4951.41/

## Подготовка данных

### Скачиваю файлы с сайта 
#### https://ftp.rcsb.org/pub/pdb/data/structures/divided/pdb/

In [2]:
import re
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
#заходим на сайт и скачиваем файлы в директорию
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
driver=webdriver.Chrome(webdriver_dir)
driver.implicitly_wait(2)

<ipython-input-3-26d21c654594>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(webdriver_dir)


In [ ]:
# захожу на страницу с каталогом файлов
driver.get("https://ftp.rcsb.org/pub/pdb/data/structures/divided/pdb/")

In [ ]:
import random
# двойной клик на "Last modified", чтобы отсортировать листы , чтобы сверху были самые новые 
# (т.к. в более новых больше вероятности, что в файле описана вторичная структура)
fld=driver.find_element_by_xpath("/html/body/pre/a[2]")
fld.click()
fld=driver.find_element_by_xpath("/html/body/pre/a[2]")
fld.click()
sleep(1.0)

list_count = 200 #сколько листов открыть  цель от 100х100
inlist_count = 100  #сколько максимум файлов в каждом листе скачать

random.seed(32)
# пройду циклом по листам каталога
#for i in random.sample(range(7,7+list_count), k=list_count):
for i in range(7,7+2*list_count-1,2):

    print("pre a",i-6)
    #print('/html/body/pre/a['+str(i)+']')
    fld=driver.find_element_by_xpath('/html/body/pre/a['+str(i)+']')
    fld.click()
    sleep(1.0)
    
    # определю сколько .gz файлов в листе
    all_page=BeautifulSoup(driver.page_source,"lxml")
    a_elements=all_page.body.pre.find_all('a')
    count_list_elements = 0
    for e in a_elements:
        if e.text.endswith('.gz'):
            #print(e.text)
            count_list_elements = count_list_elements + 1
            #print(count_list_elements)

    #print(inlist_count)
    inlist_count = min(inlist_count,count_list_elements)
    print(inlist_count)
    
    #захожу в лист и скачиваю файлы
    for j in range(7, 7+2*inlist_count-1,2):
        #print('/html/body/pre/a['+str(j)+']')
        fld=driver.find_element_by_xpath('/html/body/pre/a['+str(j)+']')
        fld.click()
        sleep(0.5)
        
    fld=driver.find_element_by_xpath('/html/body/pre/a[5]')
    fld.click()
    sleep(1.0)
    
    fld=driver.find_element_by_xpath("/html/body/pre/a[2]")
    fld.click()
    fld=driver.find_element_by_xpath("/html/body/pre/a[2]")
    fld.click()
    
    sleep(0.005)
    

### Перебрасываю файлы из загрузок в специальную директорию, разархивирую, делаю парсинг файлов

In [3]:
import shutil
import os
import gzip
import pandas as pd
import numpy as np

In [5]:
#разархивировать gz файл

def un_gz(file_name):
    """ungz zip file"""
    f_name = file_name.replace(".gz", "")
         # Получить имя файла, удалить его
    g_file = gzip.GzipFile(file_name)
         # Создать объект gzip
    open(f_name, "wb+").write(g_file.read())
       #  После открытия объекта #gzip с помощью read () он записывается в файл, созданный с помощью open ().
    g_file.close () # Закрываем объект gzip


In [6]:
import os
import shutil
protein_struct = pd.DataFrame()
#struct_content = pd.DataFrame(columns=['details','struct_marker','protein_id'])

protein_expdata = pd.DataFrame()
#protein_struct = pd.DataFrame(columns=['details','protein_id'])

download_dir= downloads_dir
target_dir='data_for_files_with_secondary_structures'

shutil.rmtree(target_dir) #туду: сделать условие: удалять папку только если она существует
os.mkdir(target_dir)

In [7]:
len(os.listdir(download_dir))

20181

In [8]:
for filename in os.listdir(download_dir):
    if (filename.startswith('pdb') and filename.endswith('ent.gz') and len(filename) == 14):
        filefrom = str(download_dir+'\\'+filename)
        fileto = str(target_dir+'\\'+filename)
        shutil.copyfile(filefrom,fileto)
        #os.remove(filefrom)
        
        un_gz(fileto)
        os.remove(fileto)
        
      #  with open('data\pdb1v70.ent') as f:
      #      contents = f.read()
      #      print(contents)
      #      
        #print(fileto)
        
        filetoparse=fileto[:-3]
        
        with open(filetoparse) as file:
            lines = [line.rstrip() for line in file]

            df_content = pd.DataFrame(lines,columns=['details'])
            
            df_content['expdta_marker'] = df_content['details'].str.startswith('EXPDTA') 
            protein_content = df_content[df_content['expdta_marker']]
            if  (len(protein_content) > 0):
                protein_content['protein_id'] =  filetoparse[-8:-4]
                protein_expdata = pd.concat([protein_expdata,protein_content])
            
            df_content['struct_marker'] = df_content['details'].str.startswith(('SHEET','HELIX'))
            struct_content = df_content[df_content['struct_marker']]            
            if (len(df_content) > 0):
                struct_content['protein_id'] =  filetoparse[-8:-4]
                protein_struct = pd.concat([protein_struct,struct_content])
                

<ipython-input-8-c14f27da6bcc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protein_content['protein_id'] =  filetoparse[-8:-4]
<ipython-input-8-c14f27da6bcc>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  struct_content['protein_id'] =  filetoparse[-8:-4]


In [9]:
protein_expdata['expdta'] = protein_expdata['details'].str[6:]
protein_expdata = protein_expdata.drop(columns=['details','expdta_marker'])
#protein_expdata

In [10]:
len(protein_struct)

1025447

In [11]:
len(protein_expdata)

19999

In [12]:
protein_expdata

,protein_id,expdta
21,1au0,X-RAY DIFFRACTION
16,1au1,X-RAY DIFFRACTION
21,1au2,X-RAY DIFFRACTION
21,1au3,X-RAY DIFFRACTION
21,1au4,X-RAY DIFFRACTION
...,...,...
19,6tbb,X-RAY DIFFRACTION
19,6tbc,X-RAY DIFFRACTION
18,6tbd,X-RAY DIFFRACTION
67,7sg0,X-RAY DIFFRACTION


In [13]:
protein_struct_pripas = protein_struct

In [14]:
protein_struct =  protein_struct_pripas
protein_struct.head(2)

,details,expdta_marker,struct_marker,protein_id
313,HELIX 1 1 TYR A 7 LYS A 10 5 ...,False,True,1au0
314,HELIX 2 2 CYS A 25 THR A 42 1 ...,False,True,1au0


In [15]:
#struct_content#[struct_content['protein_id']=='6kyi']
protein_struct['struct_'] = protein_struct['details'].str[0:6]

helices = protein_struct[protein_struct['struct_'] == 'HELIX ']
sheets = protein_struct[protein_struct['struct_'] == 'SHEET ']

helices['ser_num'] = helices['details'].str[7:10].astype(int)
helices['init_chain_id'] = helices['details'].str[19:20]
helices['init_seq_num'] = helices['details'].str[21:25].astype(int)
helices['init_seq_name'] = helices['details'].str[15:18]
helices['term_seq_num'] = helices['details'].str[33:37].astype(int)
helices['term_seq_name'] = helices['details'].str[27:30]
helices['struct_class'] = helices['details'].str[38:40].astype(int)
helices['struct_length'] = helices['details'].str[71:76]
helices['init_chain_id2'] = helices['details'].str[31:32]
helices['protein_chain_id'] = helices['protein_id']+'_'+helices['init_chain_id']
helices = helices.drop(columns=['struct_marker','expdta_marker','details'])
helices = pd.DataFrame(helices)


sheets['ser_num'] = sheets['details'].str[7:10].astype(int)
sheets['init_chain_id'] = sheets['details'].str[21:22]
sheets['init_seq_num'] = sheets['details'].str[22:26].astype(int)
sheets['init_seq_name'] = sheets['details'].str[17:20]
sheets['term_seq_num'] = sheets['details'].str[33:37].astype(int)
sheets['term_seq_name'] = sheets['details'].str[28:31]
sheets['struct_class'] = 'sh'
sheets['struct_length'] = '0'
sheets['init_chain_id2'] = sheets['details'].str[32:33]
sheets['protein_chain_id'] = sheets['protein_id']+'_'+sheets['init_chain_id']
sheets = sheets.drop(columns=['struct_marker','expdta_marker','details'])
sheets = pd.DataFrame(sheets)


#protein_struct['ser_num'] = protein_struct['details'].str[7:10].astype(int)
#protein_struct['init_chain_id'] = protein_struct['details'].str[19:20]
#protein_struct['init_seq_num'] = protein_struct['details'].str[21:25].astype(int)
#protein_struct['init_seq_name'] = protein_struct['details'].str[15:18]
#protein_struct['term_seq_num'] = protein_struct['details'].str[33:37].astype(int)
#protein_struct['term_seq_name'] = protein_struct['details'].str[27:30]
#protein_struct['struct_class'] = protein_struct['details'].str[38:40].astype(int)
#protein_struct['struct_length'] = protein_struct['details'].str[71:76]
#protein_struct['init_chain_id2'] = protein_struct['details'].str[31:32]
#protein_struct['protein_chain_id'] = protein_struct['protein_id']+'_'+protein_struct['init_chain_id']
#protein_struct = protein_struct.drop(columns=['struct_marker','expdta_marker','details'])
protein_struct = pd.concat([helices, sheets], ignore_index=True)
#protein_struct = pd.DataFrame(protein_struct)

<ipython-input-15-556fd1438a4d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  helices['ser_num'] = helices['details'].str[7:10].astype(int)
<ipython-input-15-556fd1438a4d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  helices['init_chain_id'] = helices['details'].str[19:20]
<ipython-input-15-556fd1438a4d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [16]:
#sheets

In [17]:
protein_struct = protein_struct.join(protein_expdata.set_index('protein_id'), on='protein_id')

In [18]:
protein_struct

,protein_id,struct_,ser_num,init_chain_id,init_seq_num,init_seq_name,term_seq_num,term_seq_name,struct_class,struct_length,init_chain_id2,protein_chain_id,expdta
0,1au0,HELIX,1,A,7,TYR,10,LYS,5,4,A,1au0_A,X-RAY DIFFRACTION
1,1au0,HELIX,2,A,25,CYS,42,THR,1,18,A,1au0_A,X-RAY DIFFRACTION
2,1au0,HELIX,3,A,50,PRO,56,CYS,1,7,A,1au0_A,X-RAY DIFFRACTION
3,1au0,HELIX,4,A,62,GLY,64,GLY,5,3,A,1au0_A,X-RAY DIFFRACTION
4,1au0,HELIX,5,A,68,MET,77,LYS,1,10,A,1au0_A,X-RAY DIFFRACTION
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025442,7sg1,SHEET,4,J,190,LEU,191,LYS,sh,0,J,7sg1_J,X-RAY DIFFRACTION
1025443,7sg1,SHEET,1,J,177,LYS,179,VAL,sh,0,J,7sg1_J,X-RAY DIFFRACTION
1025444,7sg1,SHEET,2,J,168,VAL,174,VAL,sh,0,J,7sg1_J,X-RAY DIFFRACTION
1025445,7sg1,SHEET,3,J,220,HIS,227,PHE,sh,0,J,7sg1_J,X-RAY DIFFRACTION


In [ ]:
#очищаю dowloads от всех pbd*.gz файлов    
def clear_dir_from_gz(download_dir):
    filename_list = []
    for filename in os.listdir(download_dir):
        if (filename.startswith('pdb') and filename.endswith('.gz')):
            filename_list.append(filename)
    print(filename_list)

    for filename in filename_list:
        #print(download_dir+filename)
        os.remove(download_dir+"\\"+filename)
        


In [ ]:
download_dir=downloads_dir
clear_dir_from_gz(download_dir)

### Подгрузка аминокислотной последовательности

In [19]:
filetoparse = acids_sequences #'C:\\Users\\elena.khotlyannik\\Documents\\HSE\\project\\pdb_seqres.txt'
with open(filetoparse) as file:
    lines = [line.rstrip() for line in file]
    
molecules = pd.DataFrame(lines)

In [20]:
molecules[1::2]
molecules[2::2]
molecule_info = pd.DataFrame(np.hstack((molecules[1::2],molecules[2::2])))

molecule_info['mol_splited'] = molecule_info[0].str.split(' ')
molecule_info['mol_type'] = molecule_info['mol_splited'].str[1].str[4:]
molecule_info = molecule_info[molecule_info['mol_type']=='protein']
molecule_info['protein_id'] = molecule_info['mol_splited'].str[0].str[1:5]

molecule_info['protein_chain_id'] = molecule_info['mol_splited'].str[0].str[1:]
molecule_info['protein_chain_length'] = molecule_info['mol_splited'].str[2].str[7:].astype(int)
molecule_info['protein_chain_sequence'] = molecule_info[1].str.lstrip() 

molecule_info = molecule_info.drop(columns=[0,1,'mol_splited'])
#molecule_info[molecule_info['protein_id']=='1dd1'].unique()

In [21]:
molecule_info[molecule_info['protein_id']=='1jnb']#.protein_chain_sequence.unique()

,mol_type,protein_id,protein_chain_id,protein_chain_length,protein_chain_sequence
22202,protein,1jnb,1jnb_A,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22203,protein,1jnb,1jnb_B,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22204,protein,1jnb,1jnb_C,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22205,protein,1jnb,1jnb_D,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22206,protein,1jnb,1jnb_E,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22207,protein,1jnb,1jnb_F,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22208,protein,1jnb,1jnb_G,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22209,protein,1jnb,1jnb_H,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22210,protein,1jnb,1jnb_I,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...
22211,protein,1jnb,1jnb_J,309,MARKRSNTYRSINEIQRQKRNRWFIHYLNYLQSLAYQLFEWENLPP...


In [22]:
protein_struct = protein_struct.drop(columns='protein_id').join(molecule_info.set_index('protein_chain_id'), on='protein_chain_id')

In [23]:
protein_struct

,struct_,ser_num,init_chain_id,init_seq_num,init_seq_name,term_seq_num,term_seq_name,struct_class,struct_length,init_chain_id2,protein_chain_id,expdta,mol_type,protein_id,protein_chain_length,protein_chain_sequence
0,HELIX,1,A,7,TYR,10,LYS,5,4,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...
1,HELIX,2,A,25,CYS,42,THR,1,18,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...
2,HELIX,3,A,50,PRO,56,CYS,1,7,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...
3,HELIX,4,A,62,GLY,64,GLY,5,3,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...
4,HELIX,5,A,68,MET,77,LYS,1,10,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025442,SHEET,4,J,190,LEU,191,LYS,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...
1025443,SHEET,1,J,177,LYS,179,VAL,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...
1025444,SHEET,2,J,168,VAL,174,VAL,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...
1025445,SHEET,3,J,220,HIS,227,PHE,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...


In [24]:
protein_struct['struct_sequence'] = ''
for i in range(0,protein_struct.shape[0]):
    start_pos = protein_struct['init_seq_num'].iloc[i]-1
    stop_pos = protein_struct['term_seq_num'].iloc[i]
    protein_struct['struct_sequence'].iloc[i] = protein_struct.protein_chain_sequence.iloc[i][start_pos:stop_pos]
    #print(i,protein_struct['protein_chain_id'].iloc[i], start_pos, 
    #      #protein_struct.protein_chain_sequence.iloc[i], 
    #      protein_struct['struct_sequence'].iloc[i])

C:\Users\elena.khotlyannik\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [25]:
#protein_struct['expdta'].unique()
#protein_struct[protein_struct['init_chain_id'] != protein_struct['init_chain_id2']]
#drop(columns = ['init_chain_id2','struct_','mol_type','expdta'])
#protein_struct['struct_sequence'] = protein_struct.protein_chain_sequence.str[protein_struct.init_seq_num:protein_struct.term_seq_num]
#protein_struct['struct_sequence'] = protein_struct.protein_chain_sequence.str[protein_struct['init_seq_num']:]

#protein_struct['struct_sequence'] = protein_struct['protein_chain_sequence'].str[15:38]
#protein_struct = protein_struct.drop(columns = ['init_chain_id2','struct_','mol_type','expdta','init_chain_id'])#[protein_struct['protein_id']=='1dd1']
#protein_struct[protein_struct['protein_id']=='1dd1']#.protein_chain_sequence.unique()
protein_struct[protein_struct['protein_id']=='1dd3']#.protein_chain_sequence.unique()
#protein_struct
#protein_struct[protein_struct['protein_id']=='1jnb']#.protein_chain_sequence.unique()
#protein_struct[protein_struct['protein_id']=='1jnb']#['init_seq_num'].astype(int)

,struct_,ser_num,init_chain_id,init_seq_num,init_seq_name,term_seq_num,term_seq_name,struct_class,struct_length,init_chain_id2,protein_chain_id,expdta,mol_type,protein_id,protein_chain_length,protein_chain_sequence,struct_sequence
504,HELIX,1,A,2,THR,11,GLU,1,10,A,1dd3_A,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,TIDEIIEAIE
505,HELIX,2,A,14,THR,30,GLY,1,17,A,1dd3_A,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,TVSELAELVKKLEDKFG
506,HELIX,3,A,30,GLY,55,GLU,1,26,A,1dd3_A,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,GVTAAAPVAVAAAPVAGAAAGAAQEE
507,HELIX,4,A,69,ASN,82,GLY,1,14,A,1dd3_A,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,NKIQVIKVVREITG
508,HELIX,5,A,84,GLY,93,GLU,1,10,A,1dd3_A,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,GLKEAKDLVE
509,HELIX,6,A,107,SER,121,ALA,1,15,A,1dd3_A,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,SKEEAEEIKKKLEEA
510,HELIX,7,B,2,THR,11,GLU,1,10,B,1dd3_B,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,TIDEIIEAIE
511,HELIX,8,B,14,THR,30,GLY,1,17,B,1dd3_B,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,TVSELAELVKKLEDKFG
512,HELIX,9,B,29,PHE,35,ALA,1,7,B,1dd3_B,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,FGVTAAA
513,HELIX,10,B,34,ALA,55,GLU,1,22,B,1dd3_B,X-RAY DIFFRACTION,protein,1dd3,128,MTIDEIIEAIEKLTVSELAELVKKLEDKFGVTAAAPVAVAAAPVAG...,AAPVAVAAAPVAGAAAGAAQEE


### Подгрузка односимвольного соответствие кодону

In [26]:
filetoparse = fileformapp #'C:\\Users\\elena.khotlyannik\\Documents\\HSE\\project\\codon mapping.txt'
with open(filetoparse) as file:
    lines = [line.rstrip() for line in file]
    
codon_mapping = pd.DataFrame(lines)

In [27]:
codon_mapping['simb3'] = codon_mapping[0].str.split().str[1].str.upper()
codon_mapping['simb1'] = codon_mapping[0].str.split().str[2].str.upper()

In [28]:
codon_mapping = codon_mapping.drop(columns=[0]).set_index('simb3')
#codon_mapping

In [29]:
protein_struct = protein_struct.join(codon_mapping, on='init_seq_name').join(codon_mapping, on='term_seq_name',lsuffix='da')

In [30]:
protein_struct

,struct_,ser_num,init_chain_id,init_seq_num,init_seq_name,term_seq_num,term_seq_name,struct_class,struct_length,init_chain_id2,protein_chain_id,expdta,mol_type,protein_id,protein_chain_length,protein_chain_sequence,struct_sequence,simb1da,simb1
0,HELIX,1,A,7,TYR,10,LYS,5,4,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,YRKK,Y,K
1,HELIX,2,A,25,CYS,42,THR,1,18,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,CWAFSSVGALEGQLKKKT,C,T
2,HELIX,3,A,50,PRO,56,CYS,1,7,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,PQNLVDC,P,C
3,HELIX,4,A,62,GLY,64,GLY,5,3,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,GCG,G,G
4,HELIX,5,A,68,MET,77,LYS,1,10,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,MTNAFQYVQK,M,K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025442,SHEET,4,J,190,LEU,191,LYS,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...,QP,L,K
1025443,SHEET,1,J,177,LYS,179,VAL,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...,NGK,K,V
1025444,SHEET,2,J,168,VAL,174,VAL,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...,DHVELSW,V,V
1025445,SHEET,3,J,220,HIS,227,PHE,sh,0,J,7sg1_J,X-RAY DIFFRACTION,protein,7sg1,259,SIEGRGGSGASRDHMAVISQKPSRDICQRGTSLTIQCQVDSQVTMM...,RNHFRCQV,H,F


### Отбор адектватных записей

In [31]:
#1 оставляю уникальные записи
protein_struct = protein_struct.drop_duplicates()
#2 оставлю struct в пределах цепочки протеина
protein_struct = protein_struct[(protein_struct.term_seq_num <= protein_struct.protein_chain_length) & 
                                  (protein_struct.init_seq_num <= protein_struct.protein_chain_length) & 
                                  (protein_struct.init_seq_num < protein_struct.term_seq_num) & 
                                  (protein_struct.init_seq_num > 0)]

#3 убираю struct с теоретической моделью получения структуры
protein_struct = protein_struct[protein_struct.expdta != '    SOLUTION NMR; THEORETICAL MODEL']

#4 убираю struct  с неопределенным начальным символом
protein_struct = protein_struct[protein_struct.simb1da.notnull()]
protein_struct = protein_struct[protein_struct.simb1.notnull()]

# Убираю странности в определении конечного и начального символа спирали
protein_struct = protein_struct[protein_struct.simb1da == protein_struct.struct_sequence.str[0]]
protein_struct = protein_struct[protein_struct.simb1 == protein_struct.struct_sequence.str[-1]]

# Убираю странности в длине цепочки
protein_struct = protein_struct[ ((protein_struct.struct_sequence.str.len() == protein_struct.struct_length.astype(int))
                                  & (protein_struct.struct_ == 'HELIX ')) 
                                | (protein_struct.struct_ == 'SHEET ')]

In [32]:
#protein_struct#[protein_struct.struct_length == 'None']

In [33]:
protein_struct[(protein_struct.struct_sequence == '')] 
                
#protein_struct[(protein_struct.init_seq_num > protein_struct.protein_chain_length) ]

#protein_struct[(protein_struct.term_seq_num > protein_struct.protein_chain_length) ]

#protein_struct[(protein_struct.term_seq_num > protein_struct.protein_chain_length)& (protein_struct.struct_sequence != '')]

#protein_struct[(protein_struct.term_seq_num <= protein_struct.protein_chain_length) & (protein_struct.init_seq_num <= protein_struct.protein_chain_length) & (protein_struct.init_seq_num < protein_struct.term_seq_num)]

,struct_,ser_num,init_chain_id,init_seq_num,init_seq_name,term_seq_num,term_seq_name,struct_class,struct_length,init_chain_id2,protein_chain_id,expdta,mol_type,protein_id,protein_chain_length,protein_chain_sequence,struct_sequence,simb1da,simb1


### Cбор статистик

In [34]:
len(protein_struct)

479288

In [35]:
len(protein_struct.protein_chain_id.unique())

25455

In [36]:
len(protein_struct.protein_id.unique())

10681

In [37]:
protein_struct.struct_class.unique()

array([5, 1, 4, 3, 10, 0, 'sh'], dtype=object)

In [38]:
protein_struct.struct_class.groupby(protein_struct.struct_class).count()

struct_class
0         11
1     176111
3          6
4          7
5      61106
10         1
sh    242046
Name: struct_class, dtype: int64

In [39]:
protein_struct.protein_chain_length.max(),protein_struct.protein_chain_length.min()

(3825, 4)

In [40]:
protein_struct.struct_length.astype(int).max(),protein_struct.struct_length.astype(int).min()

(122, 0)

### Выгрузка данных для colab

In [41]:
protein_struct.to_csv('protein_struct.csv')

### Подготовка данных на вход сети

In [18]:
protein_struct_loaded = pd.read_csv('protein_struct.csv')
protein_struct_loaded = protein_struct_loaded.head(1000)

C:\Users\elena.khotlyannik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,struct_,ser_num,init_chain_id,init_seq_num,init_seq_name,term_seq_num,term_seq_name,struct_class,struct_length,init_chain_id2,protein_chain_id,expdta,mol_type,protein_id,protein_chain_length,protein_chain_sequence,struct_sequence,simb1da,simb1
0,0,HELIX,1,A,7,TYR,10,LYS,5,4,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,YRKK,Y,K
1,1,HELIX,2,A,25,CYS,42,THR,1,18,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,CWAFSSVGALEGQLKKKT,C,T
2,2,HELIX,3,A,50,PRO,56,CYS,1,7,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,PQNLVDC,P,C
3,3,HELIX,4,A,62,GLY,64,GLY,5,3,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,GCG,G,G
4,4,HELIX,5,A,68,MET,77,LYS,1,10,A,1au0_A,X-RAY DIFFRACTION,protein,1au0,215,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,MTNAFQYVQK,M,K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479283,1025080,SHEET,2,B,217,LYS,221,PHE,sh,0,B,6tb6_B,X-RAY DIFFRACTION,protein,6tb6,386,MAKILCVLYDDPITGYPKSYARADVPKIDHYPGGQTAPTPKQIDFT...,KLHYF,K,F
479284,1025081,SHEET,3,B,194,HIS,198,VAL,sh,0,B,6tb6_B,X-RAY DIFFRACTION,protein,6tb6,386,MAKILCVLYDDPITGYPKSYARADVPKIDHYPGGQTAPTPKQIDFT...,HVGTV,H,V
479285,1025082,SHEET,4,B,251,VAL,254,ILE,sh,0,B,6tb6_B,X-RAY DIFFRACTION,protein,6tb6,386,MAKILCVLYDDPITGYPKSYARADVPKIDHYPGGQTAPTPKQIDFT...,VVTI,V,I
479286,1025083,SHEET,5,B,278,ALA,282,ASN,sh,0,B,6tb6_B,X-RAY DIFFRACTION,protein,6tb6,386,MAKILCVLYDDPITGYPKSYARADVPKIDHYPGGQTAPTPKQIDFT...,AYLVN,A,N


In [19]:
protein_struct_loaded.to_csv('protein_struct10000.csv')

In [21]:
protein_struct_loaded = protein_struct_loaded.head(1000)
protein_struct_loaded.to_csv('protein_struct1000.csv')

### y

In [11]:
data = protein_struct_loaded[['protein_chain_id','protein_chain_sequence','struct_class','struct_sequence','init_seq_num','term_seq_num']]


In [7]:
#data = data[(data.struct_class == 1) | (data.struct_class == 5)]

In [12]:
data.struct_class.unique()

array([5, 1, 4, 3, 10, 0, '1', '5', 'sh'], dtype=object)

In [15]:
data[data.struct_class == 1]

,protein_chain_id,protein_chain_sequence,struct_class,struct_sequence,init_seq_num,term_seq_num
1,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,CWAFSSVGALEGQLKKKT,25,42
2,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,PQNLVDC,50,56
4,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,MTNAFQYVQK,68,77
6,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,EKALKRAVAR,118,127
7,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,TSFQ,140,143
...,...,...,...,...,...,...
229370,5uj7_D,MSSRKSKSNSLIHTECLSQVQRILRERFCRQSPHSNLFGVQVQYKH...,1,DNTQIMNALQKY,407,418
229371,5uj7_D,MSSRKSKSNSLIHTECLSQVQRILRERFCRQSPHSNLFGVQVQYKH...,1,PTDVRQWATSS,422,432
229372,5uj7_F,MPHLENVVLCRESQVSILQSLFGERHHFSFPSIFIYGHTASGKTYV...,1,RESQVSILQSLFG,11,23
229374,5uj7_F,MPHLENVVLCRESQVSILQSLFGERHHFSFPSIFIYGHTASGKTYV...,1,GKTYVTQTLLKTL,42,54


In [17]:
data['h1'] = '0'
data['h1_bin'] = '0'
#data['h1_binary'] = '0'
data['h5'] = '0'
data['h5_bin'] = '0'
#data['h5_binary'] = '0'
data['sh'] = '0'
data['sh_bin'] = '0'
max_protein_length = 700 
for i in range(0,data.shape[0]):
    if ((data['struct_class'].iloc[i] == 1) | (data['struct_class'].iloc[i] == '1')):
        start_pos = data['init_seq_num'].iloc[i]-1
        stop_pos = data['term_seq_num'].iloc[i]
        data['h1'].iloc[i] = ''.ljust(min(start_pos,max_protein_length), '0').ljust(min(stop_pos,max_protein_length),'1').ljust(max_protein_length, '0')
        
    else:
        data['h1'].iloc[i] = ''.ljust(max_protein_length, '0')
        
    if  ((data['struct_class'].iloc[i] == 5) | (data['struct_class'].iloc[i] == '5')):
        start_pos = data['init_seq_num'].iloc[i]-1
        stop_pos = data['term_seq_num'].iloc[i]
        data['h5'].iloc[i] = ''.ljust(min(start_pos,max_protein_length), '0').ljust(min(stop_pos,max_protein_length), '1').ljust(max_protein_length, '0')
    else:
        data['h5'].iloc[i] = ''.ljust(max_protein_length, '0')
   
    if  data['struct_class'].iloc[i] == 'sh':
        start_pos = data['init_seq_num'].iloc[i]-1
        stop_pos = data['term_seq_num'].iloc[i]
        data['sh'].iloc[i] = ''.ljust(min(start_pos,max_protein_length), '0').ljust(min(stop_pos,max_protein_length), '1').ljust(max_protein_length, '0')
    else:
        data['sh'].iloc[i] = ''.ljust(max_protein_length, '0')
    
 
    data['h1_bin'].iloc[i] = np.array([int(x) for x in  data['h1'].iloc[i]])
    data['h5_bin'].iloc[i] = np.array([int(x) for x in  data['h5'].iloc[i]])
    data['sh_bin'].iloc[i] = np.array([int(x) for x in  data['sh'].iloc[i]])
    
    #if_struct = lambda x : 1 if (x == s) else 0
    #data['struct'].iloc[i] = np.array( np.sum(x,y) for x,y in data['h1_bin'].iloc[i] ,data['h5_bin'].iloc[i] )
  
data.head()

<ipython-input-17-825f08c49ee8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['h1'] = '0'
<ipython-input-17-825f08c49ee8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['h1_bin'] = '0'
<ipython-input-17-825f08c49ee8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

,protein_chain_id,protein_chain_sequence,struct_class,struct_sequence,init_seq_num,term_seq_num,h1,h1_bin,h5,h5_bin,sh,sh_bin
0,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,5,YRKK,7,10,0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000001111000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,CWAFSSVGALEGQLKKKT,25,42,0000000000000000000000001111111111111111110000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,PQNLVDC,50,56,0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,5,GCG,62,64,0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1au0_A,APDSVDYRKKGYVTPVKNQGQCGSCWAFSSVGALEGQLKKKTGKLL...,1,MTNAFQYVQK,68,77,0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
y_prep1 = data[['protein_chain_id','h1_bin','h5_bin','sh_bin']].groupby('protein_chain_id').sum()

In [ ]:
y = y_prep1
y['y_labels'] = y['h1_bin']
for i in range(0,y.shape[0]):
     y['y_labels'].iloc[i] = np.vstack((y['h1_bin'].iloc[i],y['h5_bin'].iloc[i],y['sh_bin'].iloc[i])).T    

y = y.drop(columns = ['h1_bin','h5_bin','sh_bin','sh_bin'])
y

### X

In [ ]:
X_prep1 = protein_struct_loaded[['protein_chain_id','protein_chain_sequence']].groupby('protein_chain_id').agg('max')
X_prep1['protein_chain_seq'] = X_prep1.protein_chain_sequence[:].str[:max_protein_length]
X_prep1 = X_prep1.drop(columns = ['protein_chain_sequence'])
X_prep1.head(6)

In [ ]:
simbs = np.sum(X_prep1.protein_chain_seq)
simbs_arr = np.array([x for x in  simbs])
simbs_unique = np.unique(simbs_arr)
simbs_unique

In [ ]:
simb_to_1 = lambda x : 1 if (x == s) else 0

X_prep2 = X_prep1
X_prep2['chain_simbs'] = X_prep2['protein_chain_seq']
X_prep2['chain_array'] = X_prep2['protein_chain_seq']
X_prep2['chain_array_for_pssm'] = X_prep2['protein_chain_seq']
for i in range(0,X_prep2.shape[0]):
  #  print(i)
    X_prep2['chain_simbs'].iloc[i] = np.array([x for x in  X_prep2.protein_chain_seq.iloc[i]]) 
    #print(i  , X_prep2.chain_simbs.iloc[i])
    arr_chain = X_prep2.chain_simbs.iloc[i]
    arr_chain_int = [0 for x in X_prep2.chain_simbs.iloc[i]]

    #print(arr_chain[0:])
    if len(arr_chain) <  max_protein_length:
            dops = [0]*(max_protein_length - len(arr_chain))
            arr_chain = np.hstack((arr_chain,dops))
            arr_chain_int = np.hstack((arr_chain_int,dops))
    #print(arr_chain)
    for s in simbs_unique:
      #  print(s)
        arrr =  np.array([ simb_to_1(x) for x in X_prep2['chain_simbs'].iloc[i]])
        #arrr = np.array([int(x) for x in  arrr])
        #print(arrr[0], type(arrr[0]))
        if len(arrr) <  max_protein_length:
            #print('not ok length')
            dops = [0]*(max_protein_length - len(arrr))
            arrr = np.hstack((arrr,dops))
        arr_chain = np.vstack((arr_chain,arrr))
        arr_chain_int = np.vstack((arr_chain_int,arrr))

       # print(arr_chain_int)

       # print(arr_chain)
   # print(arr_chain[0:])
    X_prep2['chain_array'].iloc[i] = arr_chain_int[1:]
   # X_prep2['chain_array'].iloc[i] = np.array([int(x) for x in  X_prep2['chain_array'].iloc[i]])
    #print(X_prep2['chain_array'].iloc[i].shape)
    X_prep2['chain_array_for_pssm'].iloc[i] = X_prep2['chain_array'].iloc[i]
    X_prep2['chain_array'].iloc[i] = X_prep2['chain_array'].iloc[i].T
    #print(X_prep2['chain_array'].iloc[i].shape)
X_prep2.head(2)

In [ ]:
X = X_prep2.drop(columns=['chain_simbs','chain_array_for_pssm'])

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
##enc = OneHotEncoder(handle_unknown='ignore')
#enc.fit(X_prep1)
#enc.categories_

In [ ]:
X['chain_array'][:][0][0]

In [ ]:
X

In [ ]:
y

In [ ]:
X.to_csv('X.csv')
y.to_csv('y.csv')

### PSSM

In [ ]:
PSSM =  X_prep2[['chain_simbs','chain_array_for_pssm']]
PSSM = pd.DataFrame(PSSM)

pos=0
sum_sib_row=0

PSSM.insert(1,'chain_len', PSSM['chain_simbs'])   

#добавляю столбцы A,A_chain_freq
for s in simbs_unique:   
    s_freq_column = s+'_chain_freq'
    s_rel_freq_column = s+'_chain_rel_freq'

    PSSM.insert(1, s, PSSM['chain_array_for_pssm'])
    PSSM.insert(1,s_freq_column, PSSM['chain_array_for_pssm'])        
    PSSM.insert(1,s_rel_freq_column, PSSM['chain_array_for_pssm'])   

#заполняю столбцы     
for i in range(0,PSSM.shape[0]):
    pos=0
    sum_sib_row=0
    PSSM.chain_len.iloc[i] = len(PSSM['chain_simbs'].iloc[i])
    for s in simbs_unique:   
        s_freq_column = s+'_chain_freq'
        s_rel_freq_column = s+'_chain_rel_freq'
       # PSSM.insert(1, s, PSSM['chain_array_for_pssm'])
       # PSSM.insert(1,s_row_freq_column, PSSM['chain_array_for_pssm'])        #print(i,pos)
        #print(PSSM.iloc[i])
        #print(PSSM[s].iloc[i][pos])
        PSSM[s_freq_column].iloc[i] = sum(PSSM[s].iloc[i][pos])
        PSSM[s_rel_freq_column].iloc[i] = sum(PSSM[s].iloc[i][pos]) / PSSM.chain_len.iloc[i]
        PSSM[s].iloc[i] = PSSM[s].iloc[i][pos]
        #    sum_sib_row = sum_sib_row + PSSM[s_row_freq_column].iloc[i]
        #print(sum_sib_row)
        pos=pos+1    
        
    
PSSM.head(2)

In [ ]:
for i in range(0,PSSM.shape[0]):
    #print(i)
    #print(PSSM.chain_simbs.iloc[i])
    #print(len(PSSM.chain_simbs.iloc[i]))

    sum_sib_row=0
    for s in simbs_unique:  
      #  print(s)
        s_freq_column = s+'_chain_freq'
        s_rel_freq_column = s+'_chain_rel_freq'
        sum_sib_row = sum_sib_row + PSSM[s_freq_column].iloc[i]
    PSSM[s_rel_freq_column].iloc[i] = sum_sib_row
    #print(sum_sib_row)
    
PSSM.head(2)  #Positional specific scoring matrices, пока не доделала, in progress 